In [98]:
import numpy as np
import pennylane as qml
import qiskit
from qiskit.visualization import array_to_latex
import scipy

In [99]:
def get_hamiltonian(unitary, time):
    lambdas, gamma = np.linalg.eig(unitary)
    phis = []
    for l in lambdas:
        phis.append(np.angle(l))
    big_phi = np.diag(np.array(phis))
    hermitian = (-1/time)*(np.matmul(gamma,np.matmul(big_phi,np.linalg.inv(gamma))))
    print("Check if U==exp(-ijH_t):\n",np.round(scipy.linalg.expm(-1j*hermitian*time),2))
    return qml.pauli_decompose(hermitian, pauli=True)

In [100]:
dev1 = qml.device("default.qubit",wires=1)
@qml.qnode(dev1)
def get_hadamard():
    qml.Hadamard(0)
    return qml.state()


dev2 = qml.device("default.qubit",wires=3)
@qml.qnode(dev2)
def qft_3():
    qml.Hadamard(0)
    qml.ControlledPhaseShift(np.pi*2**(1-2), wires=[0,1])
    qml.ControlledPhaseShift(np.pi*2**(1-3), wires=[0,2])
    
    qml.Hadamard(1)
    qml.ControlledPhaseShift(np.pi*2**(1-2), wires=[1,2])
    
    qml.Hadamard(2)
    
    qml.SWAP(wires=[0,2])
    return qml.state()

dev3 = qml.device("default.qubit",wires=4)
def inverse_qft():
    
    qml.SWAP(wires=[0,2])
    
    qml.Hadamard(2)
    
    qml.ControlledPhaseShift(-1*np.pi*2**(1-2), wires=[1,2])
    qml.Hadamard(1)
    
    qml.ControlledPhaseShift(-1*np.pi*2**(1-3), wires=[0,2])
    qml.ControlledPhaseShift(-1*np.pi*2**(1-2), wires=[0,1])
    qml.Hadamard(0)

@qml.qnode(dev3)
def qpe():
    for i in range(3):
        qml.Hadamard(i)
        
    # eigenvector
    qml.PauliX(3)
    
    # controlled unitaries
    base_angle = 5*np.pi/4
    qml.ControlledPhaseShift(base_angle, wires=[2,3])
    qml.ControlledPhaseShift(2*base_angle, wires=[1,3])    
    qml.ControlledPhaseShift(4*base_angle, wires=[0,3])    
    
    # inverse QFT
    
    inverse_qft()
    
    return qml.probs(wires=[0,1,2])

@qml.qnode(dev3)
def xs():
    for i in range(4):
        qml.PauliX(i)
#         qml.PauliZ(i)
    return qml.probs(wires=[0,1,2])


dev4 = qml.device("default.qubit",wires=2)
@qml.qnode(dev4)
def get_swap():
    qml.SWAP([0,1])
    return qml.state()

In [101]:
U = qml.matrix(get_hadamard)()
U = np.kron(U,U)
# U = qml.matrix(qft_3)()
# U = qml.matrix(qpe)()
# U = qml.matrix(xs)()
# U = qml.matrix(get_swap)()
U

array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5]])

In [102]:
hamiltonian = get_hamiltonian(U,0.24)
# hamiltonian = qml.pauli_decompose(U, pauli=True)
print(hamiltonian)

Check if U==exp(-ijH_t):
 [[ 0.5+0.j  0.5-0.j  0.5-0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]
 [ 0.5-0.j  0.5+0.j -0.5+0.j -0.5+0.j]
 [ 0.5-0.j -0.5-0.j -0.5-0.j  0.5+0.j]]
-6.544984694978735 * I
+ 3.272492347489368 * X(0) @ X(1)
+ 3.2724923474893686 * X(0) @ Z(1)
+ 3.2724923474893677 * Z(0) @ X(1)
+ 3.272492347489368 * Z(0) @ Z(1)


In [103]:
# !pip install simuq
paulis = []
for h in hamiltonian:
    print(h, hamiltonian[h])
    word = str(h).replace(" ",'')
    word = word.replace("(",'')
    word = word.replace(")",'')
    paulis.append((word.split("@"),hamiltonian[h]))
    print(word.split("@"))

I -6.544984694978735
['I']
X(0) @ X(1) 3.272492347489368
['X0', 'X1']
X(0) @ Z(1) 3.2724923474893686
['X0', 'Z1']
Z(0) @ X(1) 3.2724923474893677
['Z0', 'X1']
Z(0) @ Z(1) 3.272492347489368
['Z0', 'Z1']


In [104]:
paulis

[(['I'], -6.544984694978735),
 (['X0', 'X1'], 3.272492347489368),
 (['X0', 'Z1'], 3.2724923474893686),
 (['Z0', 'X1'], 3.2724923474893677),
 (['Z0', 'Z1'], 3.272492347489368)]

In [105]:
from simuq import QSystem, Qubit
n, T = 2, 1/4
qs = QSystem()
q = [Qubit(qs) for i in range(n)]
H = 0
for p in paulis:
    gates, coeff = p
    print(p)
    h_curr = coeff*q[0].I
    for gate in gates:
        if "Z" in gate:
            h_curr = h_curr * q[int(gate[1])].Z
        elif "X" in gate:
            h_curr = h_curr * q[int(gate[1])].X
        elif "Y" in gate:
            h_curr = h_curr * q[int(gate[1])].Y
    H = H + (h_curr)
print(H)



(['I'], -6.544984694978735)
(['X0', 'X1'], 3.272492347489368)
(['X0', 'Z1'], 3.2724923474893686)
(['Z0', 'X1'], 3.2724923474893677)
(['Z0', 'Z1'], 3.272492347489368)


In [106]:
# !pip install simuq

In [107]:
from qutip import Qobj, tensor

# minus = Qobj([[0], [1]])
# initial_state = tensor([minus] * 2)
initial_state = tensor(Qobj([[0],[1],[0],[0]]))
# initial_state = Qobj([[1], [0]])
initial_state

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]]

In [108]:
qs.add_evolution(H, T)

In [109]:
from simuq.qutip import QuTiPProvider
qpp = QuTiPProvider()
qpp.compile(qs,initial_state=initial_state)
qpp.run()
qpp.results()

Compiled.
Solved.


{'00': 0.2489326376635191,
 '01': 0.2532020870094425,
 '10': 0.24893263766351906,
 '11': 0.2489326376635192}

In [110]:
# !pip install qutip

In [96]:
# !pip install amazon-braket-sdk

In [97]:
# np.linalg.inv(gamma)